<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install autorch > log.txt
!pip install optuna > log.txt

In [44]:
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import optuna

# LOAD DATA

In [45]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')

In [46]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/icg_train.csv',index_col=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0)
allidx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(allidx)

1296

# INPUT

In [47]:
# icg
icg_input = icg_df.loc[allidx,icg_c['x']]
icg_input = icg_input.join(c620_df.loc[allidx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
icg_input = icg_input.join(c660_df.loc[allidx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
icg_input = icg_input.join(c620_df.loc[allidx].filter(regex='Receiver Temp'))

# c620
c620_feed = c620_df.loc[allidx,c620_c['x41']]

# t651
t651_feed = t651_df.loc[allidx,t651_c['x41']]

# OUTPUT

In [48]:
c620_op = c620_df.loc[allidx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[allidx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[allidx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[allidx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[allidx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[allidx,c670_c['distillate_x']+c670_c['bottoms_x']]

# CONFIG

In [49]:
config = {
      # c620
      'c620_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_G.pkl',
      'c620_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_F.pkl',

      # c660
      'c660_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_G.pkl',
      'c660_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_F.pkl',

      # c670
      'c670_M':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
      
      # col_names
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      
      # Special column (0.9999 & 0.0001)
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',

      # sp
      'c620_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_wt_always_same_split_factor_dict.pkl',
      'c660_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_wt_always_same_split_factor_dict.pkl',
      'c670_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c670_wt_always_same_split_factor_dict.pkl',
          }

In [50]:
class AllSystem(object):
  def __init__(self,config):
    
    # C620 part
    self.c620_G = joblib.load(config['c620_G'])
    self.c620_F = joblib.load(config['c620_F'])
    self.c620_op_min = joblib.load(config['c620_op_min'])
    self.c620_op_max = joblib.load(config['c620_op_max'])
    
    # C660 part
    self.c660_G = joblib.load(config['c660_G'])
    self.c660_F = joblib.load(config['c660_F'])
    self.c660_op_min = joblib.load(config['c660_op_min'])
    self.c660_op_max = joblib.load(config['c660_op_max'])

    # C670 part
    self.c670_M = joblib.load(config['c670_M'])
    self.c670_op_min = joblib.load(config['c670_op_min'])
    self.c670_op_max = joblib.load(config['c670_op_max'])

    # columns name
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    
    # other infomation
    self.c620_wt_always_same_split_factor_dict = joblib.load(config['c620_wt_always_same_split_factor_dict'])
    self.c660_wt_always_same_split_factor_dict = joblib.load(config['c660_wt_always_same_split_factor_dict'])
    self.c670_wt_always_same_split_factor_dict = joblib.load(config['c670_wt_always_same_split_factor_dict'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749
  
  def inference(self,icg_input,c620_feed,t651_feed):
    idx = icg_input.index
    #c620
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    c620_op = self.c620_G.predict(c620_case.join(c620_feed))
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op))
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    #c660
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values
    c660_op = self.c660_G.predict(c660_case.join(c660_feed))
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op))
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    #c670
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
  
  def recommend(self,icg_input,c620_feed,t651_feed):
    idx = icg_input.index
    c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = self.inference(icg_input,c620_feed,t651_feed)
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = 70
    c620_op_col = c620_op.columns.tolist()
    def c620_objective(trial):
      c620_op_dict = {}
      for name in c620_op_col:
        c620_op_dict[name] = trial.suggest_uniform(name,c620_op_min[name],c620_op_max[name])
      c620_op = pd.DataFrame(c620_op_dict,index=idx)
      輸入端bz = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values[0]
      c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op))
      s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
      輸出端bz = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
      loss = (輸入端bz - 輸出端bz)**2
      return loss
    study = optuna.create_study()
    study.optimize(c620_objective, n_trials=300)
    c620_op_opt = pd.DataFrame(study.best_params,index=idx)
    c620_op_delta = c620_op_opt - c620_op
    return c620_op_delta

# 試算模式測試

In [51]:
f = AllSystem(config)
idx = np.random.choice(allidx,size=100,replace=False,p=None)
# minibatch input 
icg_input = icg_input.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]
# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f.inference(icg_input,c620_feed,t651_feed)
f.c670_M.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.994252,0.000162717,0.359099
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.995053,0.0305829,0.260844
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.982846,0.00713907,0.153695
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.998286,0.00363678,0.529511
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.99999,2.68704e-08,0.0347494
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.99999,4.95435e-09,0.034749
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


# 推薦模式測試

In [52]:
f = AllSystem(config)
icg_input = icg_input.iloc[[0]]
c620_feed = c620_feed.iloc[[0]]
t651_feed = t651_feed.iloc[[0]]
# minibatch output 
c620_op = c620_op.iloc[[0]]
c620_wt = c620_wt.iloc[[0]]
c660_op = c660_op.iloc[[0]]
c660_wt = c660_wt.iloc[[0]]
c670_op = c670_op.iloc[[0]]
c670_wt = c670_wt.iloc[[0]]
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f.recommend(icg_input,c620_feed,t651_feed)
f.c670_M.show_metrics(c620_wt,c620_wt_,e=2e-2)

[I 2021-04-19 09:03:52,000] A new study created in memory with name: no-name-de637dca-3a46-4412-a0fa-7619706cdcce
[W 2021-04-19 09:03:52,016] Trial 0 failed because of the following error: NameError("name 'c620_op_min' is not defined")
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/_optimize.py", line 217, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-50-e7bd3363a0e0>", line 102, in c620_objective
    c620_op_dict[name] = trial.suggest_uniform(name,c620_op_min[name],c620_op_max[name])
NameError: name 'c620_op_min' is not defined


NameError: ignored